In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import joblib
import os

# Carregar os dados processados
X_train = pd.read_csv("../data/processed/X_train.csv")
y_train = pd.read_csv("../data/processed/y_train.csv").values.ravel()

# Configurar o Modelo
modelo = RandomForestClassifier(
    n_estimators=100, 
    max_depth=10, 
    random_state=42, 
    class_weight="balanced"
)

# Treinar o modelo
print("Iniciando o treinamento... Isso pode levar alguns segundos.")
modelo.fit(X_train, y_train)
print("Treinamento concluído!")

# Salvar o modelo treinado
os.makedirs("../src/models", exist_ok=True)
joblib.dump(modelo, "../src/models/model.pkl")

print("Sucesso! O modelo foi salvo em src/models/model.pkl")

Iniciando o treinamento... Isso pode levar alguns segundos.
Treinamento concluído!
Sucesso! O modelo foi salvo em src/models/model.pkl
